## Import necessary packages and dataset

In [6]:
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LassoCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

### Settings and Parameters

In [7]:
DATA_PATH = "spx_data_weekly.xlsx"

WEEKS_PER_YEAR = 52
MIN_YEARS_CONTINUOUS = 5
MIN_WEEKS_CONTINUOUS = MIN_YEARS_CONTINUOUS * WEEKS_PER_YEAR

LONG_SHORT_QUANTILE = 0.20
POSITION_WEIGHT = 0.01

VAR_QUANTILE = 0.05
ROLLING_WINDOW_WEEKS = 5 * WEEKS_PER_YEAR

MAG7_TICKERS = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA', 'META', 'TSLA']

LASSO_TARGET_TICKER = 'XLK'
LASSO_CV_SPLITS = 5

## Helper Functions

In [8]:
## Data Loading
def load_panel_sheet(path: str, sheet_name: str) -> pd.DataFrame:
    """Load sheet with 2-row headers into MultiIndex DataFrame"""
    df = pd.read_excel(path, sheet_name=sheet_name, header=[0, 1], index_col=0)
    df.index = pd.to_datetime(df.index)
    df.columns.names = ['ticker', 'field']
    return df.sort_index()

def extract_field(panel: pd.DataFrame, field: str) -> pd.DataFrame:
    """Extract (date x ticker) DataFrame for a specific field"""
    return panel.xs(field, axis=1, level='field').sort_index(axis=1)

def compute_total_returns(prices: pd.DataFrame, dividend_yields: pd.DataFrame) -> pd.DataFrame:
    """
    Compute weekly total returns including dividend yield carry.
    r_t = (P_t / P_{t-1} - 1) + (dvd_yld_{t-1} / 100) / 52
    """
    price_returns = prices.pct_change()
    dividend_carry = (dividend_yields.shift(1) / 100.0) / WEEKS_PER_YEAR
    return price_returns + dividend_carry

## Data Filtering
def longest_continuous_run(series: pd.Series) -> int:
    """Find longest consecutive run of non-null values"""
    is_valid = series.notna().to_numpy()
    if not is_valid.any():
        return 0
    
    padded = np.concatenate([[False], is_valid, [False]])
    boundaries = np.where(padded[1:] != padded[:-1])[0]
    run_lengths = boundaries[1::2] - boundaries[::2]
    return int(run_lengths.max()) if len(run_lengths) > 0 else 0

def filter_tickers_by_history(prices: pd.DataFrame, min_weeks: int) -> list:
    """Keep tickers with at least min_weeks of continuous price data"""
    valid_tickers = []
    for ticker in prices.columns:
        if longest_continuous_run(prices[ticker]) >= min_weeks:
            valid_tickers.append(ticker)
    return valid_tickers

## Portfolio Construction
def build_portfolio_weights(signal: pd.DataFrame, quantile: float, 
                           weight: float, long_short: bool) -> pd.DataFrame:
    """
    Build portfolio weights based on signal ranking.
    Long top quantile, optionally short bottom quantile.
    """
    weights = pd.DataFrame(0.0, index=signal.index, columns=signal.columns)
    
    for date in signal.index:
        cross_section = signal.loc[date].dropna()
        n_stocks = len(cross_section)
        
        if n_stocks == 0:
            continue
        
        n_select = int(np.floor(quantile * n_stocks))
        if n_select < 1:
            continue
        
        ranked = cross_section.sort_values(ascending=False)
        long_names = ranked.head(n_select).index
        weights.loc[date, long_names] = weight
        
        if long_short:
            short_names = ranked.tail(n_select).index
            weights.loc[date, short_names] = -weight
    
    return weights

def backtest_portfolio(weights: pd.DataFrame, returns: pd.DataFrame) -> pd.Series:
    """
    Calculate portfolio returns using weights from t-1 and returns at t.
    R_t = sum_i w_{t-1,i} * r_{t,i}
    """
    common_tickers = weights.columns.intersection(returns.columns)
    aligned_weights = weights[common_tickers]
    aligned_returns = returns[common_tickers]
    
    portfolio_returns = (aligned_weights.shift(1) * aligned_returns).sum(axis=1)
    return portfolio_returns.dropna()

## Performance Metrics
def calculate_performance_metrics(returns: pd.Series, risk_free: pd.Series = None) -> dict:
    """Calculate comprehensive performance metrics"""
    returns_clean = returns.dropna()
    
    if risk_free is not None:
        rf_aligned = risk_free.reindex(returns_clean.index)
        excess_returns = (returns_clean - rf_aligned).dropna()
        returns_clean = returns_clean.reindex(excess_returns.index)
    else:
        excess_returns = returns_clean
    
    # Annualized metrics
    ann_mean = excess_returns.mean() * WEEKS_PER_YEAR
    ann_vol = excess_returns.std() * np.sqrt(WEEKS_PER_YEAR)
    ann_sharpe = ann_mean / ann_vol if ann_vol > 0 else np.nan
    
    # Tail risk metrics
    skewness = excess_returns.skew()
    var_5 = np.quantile(excess_returns, VAR_QUANTILE)
    cvar_5 = excess_returns[excess_returns <= var_5].mean()
    
    # Max drawdown
    cumulative_wealth = (1 + returns_clean).cumprod()
    running_max = cumulative_wealth.cummax()
    drawdowns = (cumulative_wealth - running_max) / running_max
    max_drawdown = drawdowns.min()
    
    return {
        'ann_mean': ann_mean,
        'ann_vol': ann_vol,
        'ann_sharpe': ann_sharpe,
        'skewness': skewness,
        'VaR_5%': var_5,
        'CVaR_5%': cvar_5,
        'max_drawdown': max_drawdown
    }

def create_performance_table(strategies: dict, risk_free: pd.Series) -> pd.DataFrame:
    """Create performance comparison table for multiple strategies"""
    results = []
    for name, returns in strategies.items():
        metrics = calculate_performance_metrics(returns, risk_free)
        metrics['strategy'] = name
        results.append(metrics)
    
    return pd.DataFrame(results).set_index('strategy')

## Factor Regressions
def linear_factor_decomposition_univariate(y: pd.Series, x: pd.Series) -> dict:
    """Univariate LFD: y = alpha + beta*x + epsilon"""
    data = pd.concat({'y': y, 'x': x}, axis=1).dropna()
    X = sm.add_constant(data['x'])
    model = sm.OLS(data['y'], X).fit()
    
    return {
        'alpha_ann': model.params['const'] * WEEKS_PER_YEAR,
        'beta': model.params['x'],
        'r2': model.rsquared
    }

def linear_factor_decomposition_multivariate(y: pd.Series, X: pd.DataFrame) -> dict:
    """Multivariate LFD: y = alpha + B'X + epsilon"""
    data = pd.concat([y.rename('y'), X], axis=1).dropna()
    y_vec = data['y']
    X_mat = sm.add_constant(data.drop(columns=['y']))
    model = sm.OLS(y_vec, X_mat).fit()
    
    alpha_ann = model.params['const'] * WEEKS_PER_YEAR
    betas = model.params.drop('const')
    
    return {
        'alpha_ann': alpha_ann,
        'betas': betas,
        'r2': model.rsquared
    }

def calculate_correlation_matrix(series_dict: dict) -> pd.DataFrame:
    """Calculate correlation matrix for multiple return series"""
    df = pd.concat(series_dict, axis=1).dropna()
    df.columns = list(series_dict.keys())
    return df.corr()

## Section 1

In [ ]:
# Load data
spx_names = pd.read_excel(DATA_PATH, sheet_name='spx names', index_col=0)
spx_panel = load_panel_sheet(DATA_PATH, 'spx data')

# Extract fields
prices = extract_field(spx_panel, 'PX_LAST')
dividend_yields = extract_field(spx_panel, 'EQY_DVD_YLD_IND')
pe_ratios = extract_field(spx_panel, 'pe ratio')

# Filter tickers with >= 5 years continuous data
valid_tickers = filter_tickers_by_history(prices, MIN_WEEKS_CONTINUOUS)
prices = prices[valid_tickers]
dividend_yields = dividend_yields[valid_tickers]
pe_ratios = pe_ratios[valid_tickers]

print(f"Number of tickers after 5-year filter: {len(valid_tickers)}\n")

Number of tickers after 5-year filter: 485



### 1.1

In [29]:
# Find highest/lowest yield stocks for each date
highest_yield_ticker = dividend_yields.idxmax(axis=1)
lowest_yield_ticker = dividend_yields.idxmin(axis=1)

print("\nMost frequent HIGHEST dividend yield tickers (raw):")
print(highest_yield_ticker.value_counts().head(10))

print("\nMost frequent LOWEST dividend yield tickers (raw):")
print(lowest_yield_ticker.value_counts().head(10))

# Calculate 1-year rolling average
rolling_avg_yield = dividend_yields.rolling(window=52, min_periods=52).mean()
highest_yield_1y = rolling_avg_yield.idxmax(axis=1)
lowest_yield_1y = rolling_avg_yield.idxmin(axis=1)

print("\nMost frequent HIGHEST 1-year avg dividend yield tickers:")
print(highest_yield_1y.value_counts().head(10))

print("\nMost frequent LOWEST 1-year avg dividend yield tickers:")
print(lowest_yield_1y.value_counts().head(10))


Most frequent HIGHEST dividend yield tickers (raw):
MO      127
DD       97
T        92
KDP      71
OKE      27
CTRA     27
TRGP     19
DOW      15
FANG     13
WMB      12
Name: count, dtype: int64

Most frequent LOWEST dividend yield tickers (raw):
COO     287
CI      154
NVDA     81
Name: count, dtype: int64

Most frequent HIGHEST 1-year avg dividend yield tickers:
MO      128
T       109
DD       88
KDP      60
OKE      50
TRGP     22
CTRA     14
Name: count, dtype: int64

Most frequent LOWEST 1-year avg dividend yield tickers:
COO     244
CI      146
NVDA     81
Name: count, dtype: int64


In [30]:
# Analyze whether driven by D or P
implied_dividend = (dividend_yields / 100.0) * prices
print(f'Average Price Volatility: {prices.pct_change().std().mean()}')
print(f'Average Implied Dividend Volatility: {implied_dividend.pct_change().std().mean()}')

Average Price Volatility: 0.04345133122816815
Average Implied Dividend Volatility: 0.04072029755561552


#### It suggests that yeilds changes are driven more by price movements rather than dividend changes 

### 1.2

In [33]:
# Calculate total returns
stock_returns = compute_total_returns(prices, dividend_yields)

# Build long-only portfolio weights (top 20% by dividend yield)
weights_long_only = build_portfolio_weights(
    signal=dividend_yields,
    quantile=LONG_SHORT_QUANTILE,
    weight=POSITION_WEIGHT,
    long_short=False
)

# Backtest
returns_long_only = backtest_portfolio(weights_long_only, stock_returns)
returns_long_only.name = 'LO'

returns_long_only

date
2015-07-03    0.000000
2015-07-10    0.005680
2015-07-17    0.005415
2015-07-24   -0.016078
2015-07-31    0.017678
                ...   
2025-05-30    0.012107
2025-06-06    0.006138
2025-06-13    0.001250
2025-06-20   -0.001566
2025-06-27    0.009654
Name: LO, Length: 522, dtype: float64

### 1.3

In [34]:
# Build long-short portfolio weights (long top 20%, short bottom 20%)
weights_long_short = build_portfolio_weights(
    signal=dividend_yields,
    quantile=LONG_SHORT_QUANTILE,
    weight=POSITION_WEIGHT,
    long_short=True
)

# Backtest
returns_long_short = backtest_portfolio(weights_long_short, stock_returns)
returns_long_short.name = 'LS'

returns_long_short

date
2015-07-03    0.000000
2015-07-10    0.005320
2015-07-17   -0.000810
2015-07-24   -0.005440
2015-07-31    0.006166
                ...   
2025-05-30    0.002333
2025-06-06   -0.006279
2025-06-13    0.013152
2025-06-20   -0.003044
2025-06-27   -0.015634
Name: LS, Length: 522, dtype: float64

### 1.4

In [36]:
# Load benchmark data
additional_panel = load_panel_sheet(DATA_PATH, 'additional data')
additional_prices = extract_field(additional_panel, 'PX_LAST')

spy_price = additional_prices['SPY']
shv_price = additional_prices['SHV']

spy_returns = spy_price.pct_change()
spy_returns.name = 'SPY'

risk_free_returns = shv_price.pct_change()
risk_free_returns.name = 'RF'

# Create performance table
performance_table = create_performance_table(
    {'LO': returns_long_only, 'LS': returns_long_short, 'SPY': spy_returns},
    risk_free_returns
)

print("\nPerformance Metrics (annualized, Sharpe uses excess returns over SHV):")
print(performance_table.round(4))
print()


Performance Metrics (annualized, Sharpe uses excess returns over SHV):
          ann_mean  ann_vol  ann_sharpe  skewness  VaR_5%  CVaR_5%  \
strategy                                                             
LO          0.1371   0.1630      0.8407    0.2972 -0.0277  -0.0489   
LS          0.0148   0.0955      0.1548    1.1170 -0.0185  -0.0271   
SPY         0.1222   0.1733      0.7052   -0.6057 -0.0345  -0.0571   

          max_drawdown  
strategy                
LO             -0.3542  
LS             -0.1349  
SPY            -0.3183  



## Section 2

### 2.1

In [37]:
# LFD vs SPY
lfd_lo_spy = linear_factor_decomposition_univariate(returns_long_only, spy_returns)
lfd_ls_spy = linear_factor_decomposition_univariate(returns_long_short, spy_returns)

market_exposure_df = pd.DataFrame({
    'LO': lfd_lo_spy,
    'LS': lfd_ls_spy
}).T

print("\nLinear Factor Decomposition vs SPY:")
print(market_exposure_df.round(4))

# Correlation matrix
corr_matrix = calculate_correlation_matrix({
    'LO': returns_long_only,
    'LS': returns_long_short,
    'SPY': spy_returns
})

print("\nCorrelation Matrix:")
print(corr_matrix.round(4))


Linear Factor Decomposition vs SPY:
    alpha_ann    beta      r2
LO     0.0487  0.7595  0.6529
LS     0.0374 -0.0284  0.0027

Correlation Matrix:
         LO      LS     SPY
LO   1.0000  0.5037  0.8080
LS   0.5037  1.0000 -0.0517
SPY  0.8080 -0.0517  1.0000


### 2.2

In [39]:
# Load sector data
sector_panel = load_panel_sheet(DATA_PATH, 'sector data')
sector_prices = extract_field(sector_panel, 'PX_LAST')

# Calculate sector returns (exclude SPY and SHV)
sector_returns = sector_prices.pct_change()
for exclude in ['SPY', 'SHV']:
    if exclude in sector_returns.columns:
        sector_returns = sector_returns.drop(columns=exclude)

# Align with strategy returns
sector_returns = sector_returns.loc[returns_long_only.index.intersection(sector_returns.index)]

# Multivariate LFD
lfd_lo_sectors = linear_factor_decomposition_multivariate(returns_long_only, sector_returns)
lfd_ls_sectors = linear_factor_decomposition_multivariate(returns_long_short, sector_returns)

print(f"\nLong-Only Sector LFD:")
print(f"  Alpha (annualized): {lfd_lo_sectors['alpha_ann']:.4f}")
print(f"  R-squared: {lfd_lo_sectors['r2']:.4f}")
print(f"  Top 3 sector betas:")
for sector in lfd_lo_sectors['betas'].abs().nlargest(3).index:
    print(f"    {sector}: {lfd_lo_sectors['betas'][sector]:.4f}")

print(f"\nLong-Short Sector LFD:")
print(f"  Alpha (annualized): {lfd_ls_sectors['alpha_ann']:.4f}")
print(f"  R-squared: {lfd_ls_sectors['r2']:.4f}")
print(f"  Top 3 sector betas:")
for sector in lfd_ls_sectors['betas'].abs().nlargest(3).index:
    print(f"    {sector}: {lfd_ls_sectors['betas'][sector]:.4f}")
print()


Long-Only Sector LFD:
  Alpha (annualized): 0.0683
  R-squared: 0.9311
  Top 3 sector betas:
    XLRE: 0.2540
    XLF: 0.2437
    XLU: 0.1267

Long-Short Sector LFD:
  Alpha (annualized): 0.0588
  R-squared: 0.6437
  Top 3 sector betas:
    XLK: -0.2478
    XLV: -0.2406
    XLRE: 0.2158



### 2.3

In [40]:
# Calculate beta * sigma for each sector
sector_volatility = sector_returns.std()
exposure_lo = lfd_lo_sectors['betas'] * sector_volatility
exposure_ls = lfd_ls_sectors['betas'] * sector_volatility

print("\nLargest sector exposures (|beta_i * sigma_i|) - Long-Only:")
print(exposure_lo.abs().nlargest(5).round(4))

print("\nLargest sector exposures (|beta_i * sigma_i|) - Long-Short:")
print(exposure_ls.abs().nlargest(5).round(4))
print()


Largest sector exposures (|beta_i * sigma_i|) - Long-Only:
XLRE    0.0076
XLF     0.0076
XLE     0.0050
XLU     0.0034
XLK     0.0030
dtype: float64

Largest sector exposures (|beta_i * sigma_i|) - Long-Short:
XLK     0.0075
XLRE    0.0064
XLF     0.0064
XLV     0.0056
XLE     0.0045
dtype: float64



### 2.4

In [41]:
# Build MAG7 portfolio
available_mag7 = [t for t in MAG7_TICKERS if t in stock_returns.columns]
if len(available_mag7) < len(MAG7_TICKERS):
    missing = set(MAG7_TICKERS) - set(available_mag7)
    print(f"\nWarning: Missing MAG7 tickers: {missing}")

mag7_returns = stock_returns[available_mag7].mean(axis=1)
mag7_returns.name = 'MAG'

# Two-factor model (SPY and MAG7)
two_factors = pd.concat([spy_returns, mag7_returns], axis=1).dropna()
two_factors.columns = ['SPY', 'MAG']

lfd_lo_spy_mag = linear_factor_decomposition_multivariate(returns_long_only, two_factors)
lfd_ls_spy_mag = linear_factor_decomposition_multivariate(returns_long_short, two_factors)

print("\nLong-Only Two-Factor LFD (SPY & MAG):")
print(f"  Alpha (annualized): {lfd_lo_spy_mag['alpha_ann']:.4f}")
print(f"  Beta (SPY): {lfd_lo_spy_mag['betas']['SPY']:.4f}")
print(f"  Beta (MAG): {lfd_lo_spy_mag['betas']['MAG']:.4f}")
print(f"  R-squared: {lfd_lo_spy_mag['r2']:.4f}")

print("\nLong-Short Two-Factor LFD (SPY & MAG):")
print(f"  Alpha (annualized): {lfd_ls_spy_mag['alpha_ann']:.4f}")
print(f"  Beta (SPY): {lfd_ls_spy_mag['betas']['SPY']:.4f}")
print(f"  Beta (MAG): {lfd_ls_spy_mag['betas']['MAG']:.4f}")
print(f"  R-squared: {lfd_ls_spy_mag['r2']:.4f}")


Long-Only Two-Factor LFD (SPY & MAG):
  Alpha (annualized): 0.1246
  Beta (SPY): 1.1833
  Beta (MAG): -0.3349
  R-squared: 0.7656

Long-Short Two-Factor LFD (SPY & MAG):
  Alpha (annualized): 0.1016
  Beta (SPY): 0.3300
  Beta (MAG): -0.2833
  R-squared: 0.2381


## Section 3

In [42]:
def dynamic_sector_hedge(strategy_returns: pd.Series, factors: pd.DataFrame, 
                        window: int) -> pd.Series:
    """Apply rolling sector hedge to strategy returns"""
    data = pd.concat([strategy_returns.rename('y'), factors], axis=1).dropna()
    y = data['y']
    X = data.drop(columns=['y'])
    
    # Store rolling betas
    betas = pd.DataFrame(index=data.index, columns=X.columns, dtype=float)
    
    # Rolling regression
    for i in range(window - 1, len(data)):
        window_dates = data.index[i - window + 1:i + 1]
        y_window = y.loc[window_dates]
        X_window = X.loc[window_dates]
        X_with_const = sm.add_constant(X_window)
        model = sm.OLS(y_window, X_with_const).fit()
        betas.loc[data.index[i], :] = model.params.drop('const')
    
    # Apply hedge (use t-1 betas for time t)
    hedge = (betas.shift(1) * X).sum(axis=1)
    hedged = (y - hedge).dropna()
    return hedged

# Apply dynamic hedging
returns_lo_hedged = dynamic_sector_hedge(returns_long_only, sector_returns, ROLLING_WINDOW_WEEKS)
returns_ls_hedged = dynamic_sector_hedge(returns_long_short, sector_returns, ROLLING_WINDOW_WEEKS)

# Section 3.1 - Hedged performance
print("\n3.1 - Performance of Sector-Hedged Strategies:")
hedged_performance = create_performance_table(
    {'LO_HEDGED': returns_lo_hedged, 'LS_HEDGED': returns_ls_hedged},
    risk_free_returns
)
print(hedged_performance.round(4))

# Section 3.2 - Sector LFD of hedged strategies
print("\n3.2 - Sector LFD of Hedged Strategies:")
lfd_lo_hedged = linear_factor_decomposition_multivariate(returns_lo_hedged, sector_returns)
lfd_ls_hedged = linear_factor_decomposition_multivariate(returns_ls_hedged, sector_returns)

print(f"\nLO_HEDGED:")
print(f"  Alpha (annualized): {lfd_lo_hedged['alpha_ann']:.4f}")
print(f"  R-squared: {lfd_lo_hedged['r2']:.4f}")

print(f"\nLS_HEDGED:")
print(f"  Alpha (annualized): {lfd_ls_hedged['alpha_ann']:.4f}")
print(f"  R-squared: {lfd_ls_hedged['r2']:.4f}")
print()


3.1 - Performance of Sector-Hedged Strategies:
           ann_mean  ann_vol  ann_sharpe  skewness  VaR_5%  CVaR_5%  \
strategy                                                              
LO_HEDGED    0.0959   0.1756      0.5464    0.5096 -0.0265  -0.0545   
LS_HEDGED    0.0097   0.0995      0.0971    1.4811 -0.0176  -0.0267   

           max_drawdown  
strategy                 
LO_HEDGED       -0.3542  
LS_HEDGED       -0.1349  

3.2 - Sector LFD of Hedged Strategies:

LO_HEDGED:
  Alpha (annualized): 0.0414
  R-squared: 0.7749

LS_HEDGED:
  Alpha (annualized): 0.0379
  R-squared: 0.3804



## Section 4

In [43]:
def build_forecasts(signal: pd.DataFrame, quantile: float, 
                    long_fcst: float, short_fcst: float) -> pd.DataFrame:
    """Build fixed forecasts based on signal ranking"""
    forecasts = pd.DataFrame(0.0, index=signal.index, columns=signal.columns)
    
    for date in signal.index:
        cross_section = signal.loc[date].dropna()
        n_stocks = len(cross_section)
        if n_stocks == 0:
            continue
        
        n_select = int(np.floor(quantile * n_stocks))
        if n_select < 1:
            continue
        
        ranked = cross_section.sort_values(ascending=False)
        forecasts.loc[date, ranked.head(n_select).index] = long_fcst
        forecasts.loc[date, ranked.tail(n_select).index] = short_fcst
    
    return forecasts

def evaluate_forecasts(forecasts: pd.DataFrame, realized: pd.DataFrame) -> dict:
    """Evaluate forecast quality using pooled R2 and correlation"""
    common_idx = forecasts.index.intersection(realized.index)
    common_cols = forecasts.columns.intersection(realized.columns)
    
    fcst = forecasts.loc[common_idx, common_cols].values
    real = realized.loc[common_idx, common_cols].values
    
    mask = np.isfinite(fcst) & np.isfinite(real)
    fcst_vec = fcst[mask]
    real_vec = real[mask]
    
    if len(real_vec) == 0:
        return {'oos_r2': np.nan, 'correlation': np.nan}
    
    mse = np.mean((real_vec - fcst_vec) ** 2)
    var = np.mean((real_vec - real_vec.mean()) ** 2)
    oos_r2 = 1.0 - (mse / var) if var > 0 else np.nan
    
    correlation = np.corrcoef(fcst_vec, real_vec)[0, 1] if fcst_vec.std() > 0 else np.nan
    
    return {'oos_r2': oos_r2, 'correlation': correlation}

# Build forecasts (+0.1% for longs, -0.1% for shorts)
forecasts = build_forecasts(
    signal=dividend_yields,
    quantile=LONG_SHORT_QUANTILE,
    long_fcst=0.001,
    short_fcst=-0.001
)

# Section 4.1 - Forecasts vs realized returns
realized_forward = stock_returns.shift(-1)  # Next period realized returns
eval_41 = evaluate_forecasts(forecasts, realized_forward)

print("\n4.1 - Forecast vs Realized Returns:")
print(f"  Out-of-sample R²: {eval_41['oos_r2']:.4f}")
print(f"  Correlation: {eval_41['correlation']:.4f}")

# Section 4.2 - Forecasts vs SPY-hedged residuals
def calculate_rolling_residuals_single_factor(returns: pd.DataFrame, 
                                              factor: pd.Series, 
                                              window: int) -> pd.DataFrame:
    """Calculate rolling residuals for each stock vs single factor"""
    idx = returns.index.intersection(factor.index)
    residuals = pd.DataFrame(index=idx, columns=returns.columns, dtype=float)
    
    for ticker in returns.columns:
        data = pd.concat([returns[ticker], factor], axis=1).dropna()
        data.columns = ['y', 'f']
        
        if len(data) < window:
            continue
        
        # Rolling regression
        X_with_const = sm.add_constant(data['f'])
        rolling_model = RollingOLS(data['y'], X_with_const, window=window).fit()
        
        # Forward residual: realized(t+1) - predicted(t+1) using betas(t)
        y_forward = data['y'].shift(-1)
        f_forward = data['f'].shift(-1)
        predicted = rolling_model.params['const'] + rolling_model.params['f'] * f_forward
        
        resid_forward = y_forward - predicted
        residuals.loc[resid_forward.index, ticker] = resid_forward
    
    return residuals

spy_hedged_residuals = calculate_rolling_residuals_single_factor(
    stock_returns, spy_returns, ROLLING_WINDOW_WEEKS
)
eval_42 = evaluate_forecasts(forecasts, spy_hedged_residuals)

print("\n4.2 - Forecast vs SPY-Hedged Residuals:")
print(f"  Out-of-sample R²: {eval_42['oos_r2']:.4f}")
print(f"  Correlation: {eval_42['correlation']:.4f}")

# Section 4.3 - Forecasts vs sector-hedged residuals
def calculate_rolling_residuals_multi_factor(returns: pd.DataFrame, 
                                             factors: pd.DataFrame, 
                                             window: int) -> pd.DataFrame:
    """Calculate rolling residuals for each stock vs multiple factors"""
    idx = returns.index.intersection(factors.index)
    residuals = pd.DataFrame(index=idx, columns=returns.columns, dtype=float)
    
    for ticker in returns.columns:
        data = pd.concat([returns[ticker].rename('y'), factors], axis=1).dropna()
        
        if len(data) < window:
            continue
        
        y = data['y']
        X = data.drop(columns=['y'])
        X_with_const = sm.add_constant(X)
        
        # Rolling regression
        rolling_model = RollingOLS(y, X_with_const, window=window).fit()
        
        # Forward residual
        y_forward = y.shift(-1)
        X_forward = X.shift(-1)
        
        predicted = rolling_model.params['const'].copy()
        for col in X.columns:
            predicted += rolling_model.params[col] * X_forward[col]
        
        resid_forward = y_forward - predicted
        residuals.loc[resid_forward.index, ticker] = resid_forward
    
    return residuals

sector_hedged_residuals = calculate_rolling_residuals_multi_factor(
    stock_returns, sector_returns, ROLLING_WINDOW_WEEKS
)
eval_43 = evaluate_forecasts(forecasts, sector_hedged_residuals)

print("\n4.3 - Forecast vs Sector-Hedged Residuals:")
print(f"  Out-of-sample R²: {eval_43['oos_r2']:.4f}")
print(f"  Correlation: {eval_43['correlation']:.4f}")


4.1 - Forecast vs Realized Returns:
  Out-of-sample R²: -0.0064
  Correlation: 0.0065

4.2 - Forecast vs SPY-Hedged Residuals:
  Out-of-sample R²: 0.0005
  Correlation: 0.0259

4.3 - Forecast vs Sector-Hedged Residuals:
  Out-of-sample R²: -0.0004
  Correlation: 0.0084


## Section 5

In [45]:
# Get sector dividend yields for forecast consistency
sector_div_yields = extract_field(sector_panel, 'EQY_DVD_YLD_IND')
sector_total_returns = compute_total_returns(sector_prices, sector_div_yields)

# Remove non-sector ETFs
for exclude in ['SPY', 'SHV']:
    if exclude in sector_total_returns.columns:
        sector_total_returns = sector_total_returns.drop(columns=exclude)

# Section 5.1 - LASSO Replication
if LASSO_TARGET_TICKER not in sector_total_returns.columns:
    print(f"\nWarning: Target {LASSO_TARGET_TICKER} not in sector data")
else:
    target_returns = sector_total_returns[LASSO_TARGET_TICKER]
    replication_universe = sector_total_returns.drop(columns=[LASSO_TARGET_TICKER])
    
    # Align and clean data
    lasso_data = pd.concat([target_returns.rename('target'), replication_universe], axis=1).dropna()
    y_lasso = lasso_data['target']
    X_lasso = lasso_data.drop(columns=['target'])
    
    # LASSO with time series cross-validation
    tscv = TimeSeriesSplit(n_splits=LASSO_CV_SPLITS)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_lasso)
    
    lasso_model = LassoCV(cv=tscv, max_iter=10000, random_state=42)
    lasso_model.fit(X_scaled, y_lasso)
    
    # Extract coefficients (unscaled)
    coef_unscaled = lasso_model.coef_ / scaler.scale_
    intercept_unscaled = lasso_model.intercept_ - np.sum((scaler.mean_ / scaler.scale_) * lasso_model.coef_)
    
    # Get non-zero weights
    replication_weights = pd.Series(coef_unscaled, index=X_lasso.columns)
    replication_weights = replication_weights[replication_weights != 0].sort_values(key=abs, ascending=False)
    
    print("\n5.1 - LASSO Replication Weights (non-zero):")
    print(replication_weights.round(4))
    
    # Build replication portfolio returns
    rep_returns = (X_lasso @ replication_weights.reindex(X_lasso.columns).fillna(0)) + intercept_unscaled
    
    # Section 5.2 - Arbitrage Check via Forecast Consistency
    # Compare dividend-yield forecasts
    sector_yields_aligned = sector_div_yields.loc[lasso_data.index]
    
    # Forecast for target (weekly carry)
    target_forecast = (sector_yields_aligned[LASSO_TARGET_TICKER] / 100.0) / WEEKS_PER_YEAR
    
    # Forecast for replication portfolio
    if len(replication_weights) > 0:
        rep_forecast = ((sector_yields_aligned[replication_weights.index] / 100.0) / WEEKS_PER_YEAR * 
                       replication_weights).sum(axis=1)
    else:
        rep_forecast = pd.Series(0, index=target_forecast.index)
    
    # Forecast spread (arbitrage indicator)
    forecast_spread = target_forecast - rep_forecast
    
    # Tracking error
    tracking_diff = y_lasso - rep_returns
    tracking_error_ann = tracking_diff.std() * np.sqrt(WEEKS_PER_YEAR)
    
    print("\n5.2 - Arbitrage Analysis:")
    print(f"  Replication R²: {lasso_model.score(X_scaled, y_lasso):.4f}")
    print(f"  Tracking error (annualized): {tracking_error_ann:.4f}")
    print(f"  Mean forecast spread (annualized): {forecast_spread.mean() * WEEKS_PER_YEAR:.4f}")
    print(f"  Forecast spread volatility (ann): {forecast_spread.std() * np.sqrt(WEEKS_PER_YEAR):.4f}")
    print(f"\n  Largest absolute forecast spreads:")
    print(forecast_spread.abs().nlargest(5).round(6))


5.1 - LASSO Replication Weights (non-zero):
XLY     0.4425
XLC     0.4064
XLI     0.3833
XLF    -0.1781
XLV     0.1495
XLE    -0.0601
XLU    -0.0543
XLB    -0.0453
XLP    -0.0312
XLRE    0.0047
dtype: float64

5.2 - Arbitrage Analysis:
  Replication R²: 0.7969
  Tracking error (annualized): 0.1102
  Mean forecast spread (annualized): 0.0025
  Forecast spread volatility (ann): 0.0004

  Largest absolute forecast spreads:
date
2020-03-20    0.000191
2020-03-27    0.000175
2020-04-03    0.000169
2020-04-24    0.000150
2020-04-17    0.000150
dtype: float64
